In [1]:
import tensorflow as tf
import numpy as np
sess=tf.InteractiveSession()

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
           "collect wood and don't assign them tasks and work, but rather "
           "teach them to long for endless immensity of the sea.")

# Multi RNN with Fully connected

In [31]:
char_list = list(set(sentence))
char_dic = {c:c_idx for c_idx, c in enumerate(char_list)}

seq_length = 12
x_data=[]
y_data=[]

for i in range(len(sentence)-seq_length):
    x_str = sentence[i:i+seq_length]
    y_str = sentence[i+1:i+1+seq_length]
    
    x_num = [char_dic[s] for s in x_str]
    y_num = [char_dic[s] for s in y_str]
    
    x_data.append(x_num)
    y_data.append(y_num)
    
with tf.variable_scope("MultiRNN_FC_no_activation_function___"):
    #parameters
    hidden_size = len(char_list)
    nb_classes = len(char_list)
    batch_size = len(x_data)
    
    X = tf.placeholder(tf.int32, [None, seq_length]) #int로 받아야 one_hot
    Y = tf.placeholder(tf.int32, [None, seq_length]) #int로 받아야 곱 가능
    # seq2seq 비용함수를 적용하려면 반드시 batch*seq 의 크기여야 함!!!!!
    x_one_hot = tf.one_hot(X, nb_classes)
    # 여러개 쌓으면 쌓을 수록 모델이 정교해짐
    cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([cell]*2, state_is_tuple=True)
    # cell 을 세로로 세개 쌓은 것과 동일
    
    # state_is_tuple 은 [1,2],[3,4]가 False 면 [1,2,3,4] True면 [[1,2],[3,4]]    
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, states = tf.nn.dynamic_rnn(cell, x_one_hot,
                   dtype=tf.float32)
    
    # softmax -> 다 같은 cell 을 통해서 나온 아이들 이기 때문에 하나의 sotfmax 사용
    # cnn은 진짜로 feature extract 이고 rnn 은 원래 결과를 도출하는데,
    # rnn 도 cnn 처럼 feature extract 한 것 처럼 생각하고 softmax 로 결과를 도출
    
    x_for_softmax = tf.reshape(outputs, [-1, hidden_size])
    softmax_w1= tf.get_variable("softmax_w1", [hidden_size, nb_classes],
            initializer = tf.contrib.layers.xavier_initializer())
    softmax_b1= tf.get_variable("sotfmax_b1", [nb_classes],
            initializer = tf.contrib.layers.xavier_initializer())
    outputs = tf.matmul(x_for_softmax,softmax_w1)+softmax_b1
    # entropy cost 함수를 구할 필요가 없기 때문에 굳이 softmax 쓰지 않음.
    outputs = tf.reshape(outputs, [batch_size, seq_length, nb_classes])
    # output의 shape와 동일 

    weights = tf.ones([batch_size, seq_length])
    seq_cost = tf.contrib.seq2seq.sequence_loss(logits=outputs,
    targets = Y, weights=weights) # activation fuction 을 거치지 않은 값으로 해야 성능이 좋음.
    cost = tf.reduce_mean(seq_cost)
    train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

In [23]:
sess =tf.Session()
sess.run(tf.global_variables_initializer()) # 변수를 할당하는 동시에 초기화 기능
for step in range(3000):
    c, p, _ = sess.run([cost, outputs, train], feed_dict={X:x_data,
                                                             Y:y_data})
    if step%300==0:
        p = np.argmax(p, axis=2)
        p_str = [char_list[c_idx] for s in p for c_idx in s]
        p_str = "".join(p_str)
        print(f"step:{step}, loss: {c}, predicted : {p_str}")


step:0, loss: 3.2466251850128174, predicted : rrrlddddllllddddddddlldddddddddlllddddddddllldddlldrrllldddddddllllddddddllllldddddllrllldddddlldlddddddddlllddddddddlllddddddddddddddddddlllllldddddlllllldlrlllllllllldllllllllddlllllllllddllddldddddddllrrrrdddddlllddddddddlllldddddddlllllddddddlllllldddddlllllllddddllllllllddlllllllllldlllllllllllrllllllllllllllllllllllldddddlldddddddrdlldddddldrrrrrrdddddlllllddddddlllldddddddllldddddddddlldddddddddllldddddddlllllddddddllllllllllllllllllddddlllllllllldlllllllllddlllllllllldllllllllllllllllllllllldllllllllllllllllllllllldrllllllllddllllllllldddllllddddddddddddddddddddddddddddddddddddddddddddddddddddddddllrrdddddddllddddddddddlddddddddddlldddddddddllldddddddllllldddddddrllllllddddrllllldddddrllllllddddrrlllllllrrrrllllllldlrllllllllllrrllllllllrllddddddllrrdddddddllrrrddddddlrrrrrdrrdrrrrrrrrllllrrrrrrrrddddrrrrrrr.dlrrrrrrrrrrlrrrrrrrrrrrlrrrrrrrrrrrlrrrrdddrrrrdddddddddlllddddddddlllldldddddlllldldddrlrrllllddrrrrrrrdddddrdlldddddldrrrrr

step:1500, loss: 0.18597814440727234, predicted : f you want toyou want totou want to  u want to bn want to butwant to buitont to builont to buildnd to build d to build a to build a to build a s  build a shnbuild a shitutld a shiputld a ship,tld a ship, fd a ship, de a ship, do a ship, dontnship, don'nship, don'tthip, don't sip, don't dep, don't drf, don't drue don't drum bon't drum ton't drum u n't drum upn't arum up dt drum up pt drum up pe trum up peotoum up peop um up peopl m up peoplet up people  tp people ttp people tot people togepeople togeteople togeteople togethnple togethenle togetheree together ectogether tntogether toto ether to   ether to cnether to conther to colnher to coll em to colleem to collecn to collect to collect to bollect w  bollect wonbollect wootollect woodtllect wood nlect wood aeect wood anect wood andnt wood and t wood and d tood and dotord and donord and don'nd and don'tn and don't  a d don't atnd don't asnd won't assd don't assi aon't assigton't dssign n

In [27]:
p = sess.run(outputs, feed_dict = {X:x_data})
p = np.argmax(p, 2)
for idx, seq in enumerate(p):
    p_str = [char_list[i] for i in seq]
    if idx == 0:
        print("".join(p_str), end="") # 첫번째 문장만 다 출력하고
    else:
        print("".join(p_str)[-1], end="") # 나머지는 끝에 하나만 출력하는 식으로 문장을 형성

t you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for endless immensity of the sea.

# Dynamic RNN

In [ ]:
sequence_length=[5,2,3] # 각각의 sequence length의 인자를 알려줌으로써 길이가 다른 input 해결 가능
# tf.nn.dynamic_rnn 의 인자
# 결과는 나머지 빈공간을 0으로 채워서 반환